In [31]:
import os
from tensorflow import keras
import tensorflow as tf
import tensorflow.lite as tflite
import numpy as np

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb

### Answer to Q1:
- **43** (closest to the actual 44.8) 

In [19]:
model = keras.models.load_model("bees-wasps.h5")

In [24]:
tflite_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()
tflite_model_name = "bees-wasps.tflite"
with open(tflite_model_name, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpmteesytw/assets


INFO:tensorflow:Assets written to: /tmp/tmpmteesytw/assets
2023-11-27 20:39:40.144246: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-27 20:39:40.144284: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-27 20:39:40.144492: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpmteesytw
2023-11-27 20:39:40.145324: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-27 20:39:40.145335: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpmteesytw
2023-11-27 20:39:40.147318: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-27 20:39:40.225256: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpmteesytw
2023-11-27 20:39:40.235540: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [25]:
# getsize returns size in bytes, need to delete by 1 000 000 to get MBs
print(os.path.getsize(tflite_model_name)/1_000_000)

44.8662


## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* 13
* 24

### Answer to Q2:
- **13**


In [26]:
interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [41]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
output_index

13

## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

In [28]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [30]:
# download image and resize it
url = "https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"
img = download_image(url)
img = prepare_image(img, (150, 150))


## Question 3

Now we need to turn the image into numpy array and pre-process it.

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3450980
* 0.5450980
* 0.7450980
* 0.9450980

### Answer to Q3:
- **0.9450980**

In [43]:
img_np_arr = np.array(img, dtype="float32")
X = np.array([img_np_arr])
# Similar to rescale in HW 8
X = X * 1./255 

In [44]:
print(X[0][0][0][0].round(7))

0.945098


## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.258
* 0.458
* 0.658
* 0.858


### Answer to Q4:
- **0.658** (closest to the actual 0.659)

In [46]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

In [48]:
preds = interpreter.get_tensor(output_index)
preds

array([[0.6592137]], dtype=float32)

## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10
COPY bees-wasps-v2.tflite .
```

And pushed it to [`agrigorev/zoomcamp-bees-wasps:v2`](https://hub.docker.com/r/agrigorev/zoomcamp-bees-wasps/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The version of Python is 3.10, so you need to use the right wheel for 
  TF-Lite. For Tensorflow 2.14.0, it's https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


## Question 5

Download the base image `agrigorev/zoomcamp-bees-wasps:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 162 Mb
* 362 Mb
* 662 Mb
* 962 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

### Answer to Q5:
- **662 Mb** (`docker images | grep "grigorev"`)


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `bees-wasps-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

What's the output from the model?

* 0.2453
* 0.4453
* 0.6453
* 0.8453

### Answer to Q6:
- **0.4453**